# [Huggingface Transformers 入門 (25) - 日本語の要約の学習](https://note.com/npaka/n/n6df9ef13a0ed)

## 2. 要約データセットの準備

In [ ]:
!git clone https://github.com/KodairaTomonori/ThreeLineSummaryDataset.git

Cloning into 'ThreeLineSummaryDataset'...
remote: Enumerating objects: 31, done.
remote: Total 31 (delta 0), reused 0 (delta 0), pack-reused 31
Unpacking objects: 100% (31/31), done.


In [ ]:
%cd ThreeLineSummaryDataset/data

/content/ThreeLineSummaryDataset/data


In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from pprint import pprint
import time


# コンテンツの取得
def get_content(id):
   # time.sleep(10)
   time.sleep(3)
   URL = 'https://news.livedoor.com/article/detail/'+id+'/'
   print(URL)
   try:
       with urlopen(URL) as res:
           # 本文
           output1 = ''
           html = res.read().decode('euc_jp', 'ignore')
           soup = BeautifulSoup(html, 'html.parser')
           lineList = soup.select('.articleBody p')
           for line in lineList:
               if len(line.contents) > 0 and type(line.contents[0]) == NavigableString:
                   output1 += line.contents[0].strip()
           if output1 == '': # 記事がない
               return
           output1 += '\n'

           # 3行要約
           output0 = ''
           summaryList = soup.select('.summaryList li')
           for summary in summaryList:
               output0 += summary.contents[0].strip()+'\t'
           if output0 == '': # 記事がない
               return

           # 出力
           print(output0+output1)
           with open('output.tsv', mode='a') as f:
               f.writelines(output0+output1)
   except Exception:
       print('Exception')

# IDリストの生成の取得
idList = []
with open('train.csv', mode='r') as f:
   lines = f.readlines()
   for line in lines:
       id = line.strip().split(',')[3].split('.')[0]
       idList.append(id)

# コンテンツの取得
# for i in range(0, 10): # 取得したい記事のINDEXをここで指定
for i in range(0, 1000): # 取得したい記事のINDEXをここで指定
   print('index:', i)
   get_content(idList[i])

index: 0
https://news.livedoor.com/article/detail/11097202/
岡むら屋から、期間限定の新メニュー「じゃが肉めし」が登場する	男爵いもなどは味噌ベースで煮こまれ、しっかり味が染み込んでいるとのこと	「岡むら屋特製肉じゃが」と言うべき一品に、仕上がっているという	新橋と秋葉原に店を構える「岡むら屋」。味噌ベースの独自の味つけで牛バラ肉を煮込んだ具材がたっぷり乗ったオリジナル丼『肉めし』で知られる店です。自分も新橋店で『肉めし』を食べたことがありますが、濃いめの味付けでトロットロに煮こまれた牛肉とホカホカご飯は最高の相性！ 味の染み込んだ豆腐も絶品で、腹ぺこボーイズ&ガールズたちの胃袋を満たし続けている丼なのです。昨年開催された『第2回全国丼グランプリ』では、肉丼部門で金賞受賞を獲得しています。そんな「岡むら屋」に期間限定新メニューが登場。その名も『じゃが肉めし』！肉と芋――このまま名作文学のタイトルにもなりそうな、重厚かつ甘美な響き。これって食いしんぼうたちにとっては定番かつ夢の組み合わせではないでしょうか。この組み合わせではすでに「肉じゃが」という殿堂入りメニューがありますが、今回の『じゃが肉めし』は、“岡むら屋特製肉じゃが”と言うべき一品に仕上がっているそうです。北海道産の男爵いもとしらたきを合わせ、味噌ベースの大鍋で他の具材と一気に煮込むことで、しっかり味が染み込んでいるとのこと。商品写真が公開されていますが、ビジュアルを見ただけで、その染みこみ具合いは一目瞭然！丼というステージで繰り広げられる、肉×芋×白米というスーパースターの競演。肉好き必食の1杯と言えそうです。あ、定食も同時販売されるので、気分で選べるのもうれしいですね。詳しい販売期間などは、店舗にお問い合わせください。肉めし「岡むら屋」

index: 1
https://news.livedoor.com/article/detail/11104959/
東京駅周辺の安くて美味しい「蕎麦ランチ」の名店を紹介している	「越後そば 東京店」では、ミニかき揚げ丼セットがおすすめと筆者	その他には、「手打ちそば 石月」「酢重正之 楽」「鎌倉 一茶庵 丸山」など	名店がひしめく「丸の内・日本橋」エリアで＜うまい蕎麦ランチ＞が食べられるお店を厳選してご紹介。

In [ ]:
file_name = "output.tsv"

In [ ]:
from google.colab import files
#files.download('output.tsv')
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 3. 要約データセットの書式の変換

In [ ]:
%cd /content/ThreeLineSummaryDataset/data

/content/ThreeLineSummaryDataset/data


In [ ]:
import pandas as pd
import numpy as np
from numpy.random import RandomState

#df = pd.read_csv('C:/Dataset.csv')
df = pd.read_table(file_name, names=('summary', 'summary2', 'summary3', 'text'))

#df['split'] = np.random.randn(df.shape[0], 1)
#msk = np.random.rand(len(df)) <= 0.7
#train = df[msk]
#test = df[~msk]

rng = RandomState()

train = df.sample(frac=0.9, random_state=rng)
dev = df.loc[~df.index.isin(train.index)]

In [ ]:
df.head(10)

,summary,summary2,summary3,text
0,岡むら屋から、期間限定の新メニュー「じゃが肉めし」が登場する,男爵いもなどは味噌ベースで煮こまれ、しっかり味が染み込んでいるとのこと,「岡むら屋特製肉じゃが」と言うべき一品に、仕上がっているという,新橋と秋葉原に店を構える「岡むら屋」。味噌ベースの独自の味つけで牛バラ肉を煮込んだ具材がたっ...
1,東京駅周辺の安くて美味しい「蕎麦ランチ」の名店を紹介している,「越後そば 東京店」では、ミニかき揚げ丼セットがおすすめと筆者,その他には、「手打ちそば 石月」「酢重正之 楽」「鎌倉 一茶庵 丸山」など,名店がひしめく「丸の内・日本橋」エリアで＜うまい蕎麦ランチ＞が食べられるお店を厳選してご紹介...
2,8日から「サンクトガーレン」は、「チョコビール」4種を販売する,ダークな茶色、ほろ苦いビターチョコのような香りが特長のビール,毎年バレンタインシーズン限定で販売され、2016年で11年目となるそう,地ビールメーカー「サンクトガーレン」は、「チョコビール」4種を2016年1月8日から全国の取...
3,1年以内に登場した、東京みやげの新定番を紹介している,カファレルの「東京ジャンドゥーヤチョコパイ」,オーボンヴュータンの「ソーシソン・セック」など,何かと人に会う機会が多いこの時期。そんな時、頭を悩ませるのが手みやげ選びだ。毎年似たようなも...
4,新しいポテトチップスの味として「パクチー味」が話題になっている,フリーズドライパクチーとパクチードレッシングがついているものもある,自宅で手軽にパクチーが存分に楽しめるようになっているという,新しいポテトチップスの味として今、話題になっているのが「それだけではありません。さらに、普通...
5,ロッテリアが2月15日から「エビバーガー」をリニューアルして販売する,エビにバナメイ種を採用し、衣も粗めに挽くことで具材の味を引き立たせる,また同日より期間限定で「桜えびタルタルのエビバーガー」の販売も開始する,「エビバーガー」は1977年の発売以来、ロッテリアの看板商品として愛され続けてきた商品だ。リ...
6,塩ラーメンがおいしい東京近郊の店を紹介している,「ジョニーとリー」の鶏ソバは絶妙なバランスと火加減で煮たスープが絶品,「MONACO」の塩そばはトリュフやポルチーニなどでスープが作られる,澄んだスープの中に、こだわりの塩や鶏、魚介などの旨みがたっぷり。ごまかしがきかない、真っ当勝...
7,岡むら屋から、期間限定の新メニュー「じゃが肉めし」が登場する,男爵いもなどは味噌ベースで煮こまれ、しっかり味が染み込んでいるとのこと,「岡むら屋特製肉じゃが」と言うべき一品に、仕上がっているという,新橋と秋葉原に店を構える「岡むら屋」。味噌ベースの独自の味つけで牛バラ肉を煮込んだ具材がたっ...
8,東京駅周辺の安くて美味しい「蕎麦ランチ」の名店を紹介している,「越後そば 東京店」では、ミニかき揚げ丼セットがおすすめと筆者,その他には、「手打ちそば 石月」「酢重正之 楽」「鎌倉 一茶庵 丸山」など,名店がひしめく「丸の内・日本橋」エリアで＜うまい蕎麦ランチ＞が食べられるお店を厳選してご紹介...
9,8日から「サンクトガーレン」は、「チョコビール」4種を販売する,ダークな茶色、ほろ苦いビターチョコのような香りが特長のビール,毎年バレンタインシーズン限定で販売され、2016年で11年目となるそう,地ビールメーカー「サンクトガーレン」は、「チョコビール」4種を2016年1月8日から全国の取...


In [ ]:
train[['text', 'summary']].to_csv('train.csv', index=False) 
dev[['text', 'summary']].to_csv('dev.csv', index=False) 

In [ ]:
from google.colab import files
files.download('train.csv')
files.download('dev.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## ​4. 日本語の要約の学習

In [ ]:
# データの永続化
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


In [ ]:
!cp -p /content/ThreeLineSummaryDataset/data/*.tsv .
!cp -p /content/ThreeLineSummaryDataset/data/*.csv .

In [ ]:
# ソースからのHuggingface Transformersのインストール
!git clone https://github.com/huggingface/transformers -b v4.4.2
!pip install -e transformers

fatal: destination path 'transformers' already exists and is not an empty directory.
Obtaining file:///content/drive/My%20Drive/work/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 870kB 11.2MB/s 
     |████████████████████████████████| 3.3MB 24.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=d9bcd0f5bcb586855b3da657549db67fe71a856c25ddbf5948bfc97ff9dc22f1
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
  Running setup.py develop for transformers


In [ ]:
# メニュー「ランタイム → ランタイムを再起動」で「Google Colab」を再起動

# 作業フォルダに戻る
%cd '/content/drive/My Drive/work/'


/content/drive/My Drive/work


In [ ]:
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1


     |████████████████████████████████| 163kB 12.6MB/s 
     |████████████████████████████████| 245kB 11.7MB/s 


In [ ]:
!pip install rouge_score
!pip install sentencepiece


     |████████████████████████████████| 1.2MB 11.1MB/s 


In [ ]:
%%time

# ファインチューニングの実行
!python ./transformers/examples/seq2seq/run_summarization.py \
    --model_name_or_path=sonoisa/t5-base-japanese \
    --do_train \
    --do_eval \
    --train_file=train.csv \
    --validation_file=dev.csv \
    --per_device_train_batch_size=2 \
    --per_device_eval_batch_size=2 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --output_dir=output/ \
    --predict_with_generate \
    --use_fast_tokenizer=False

2021-04-12 09:15:08.337780: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
04/12/2021 09:15:10 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
04/12/2021 09:15:10 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='output/', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, logging_dir='runs/Apr12_0

## 5. 日本語の要約の推論

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# モデルとトークナイザーの準備
model = AutoModelForSeq2SeqLM.from_pretrained('output/')    
tokenizer = AutoTokenizer.from_pretrained('sonoisa/t5-base-japanese') 

# テキスト
text = r"""
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女ブルマと出会う。
そこで、7つ集めると神龍が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。
人さらいのウーロンや盗賊のヤムチャなどを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。
"""

# テキストをテンソルに変換
inputs = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)

# 推論
model.eval()
with torch.no_grad():
    summary_ids = model.generate(inputs) #, max_length=512, min_length=5, length_penalty=5., num_beams=2)
    summary = tokenizer.decode(summary_ids[0])
    print(summary)


<pad><extra_id_0>ドラゴンボールを探す旅に出た孫悟空は、7つ集めると神龍が現れるというドラゴンボールを探す旅に出る。</s>
